In [1]:
#导入必要工具包
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('Diabetes2.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,125.0,33.6,0.627,50,1
1,1,85.0,66.0,29.0,125.0,26.6,0.351,31,0
2,8,183.0,64.0,29.0,125.0,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [3]:
y = data['Outcome']
x = data.drop('Outcome',axis=1)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=30)

In [4]:
from sklearn.preprocessing import StandardScaler
ss_x = StandardScaler()
x_train = ss_x.fit_transform(x_train)
x_test = ss_x.transform(x_test)

# LinearSVC

In [5]:
#还是采用交叉验证的方式来寻找最佳参数
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
penalty = ['l1','l2']
Cs = [0.01,0.1,1,10,100]
param_grid = {'penalty':penalty,'C':Cs}
lsvc = LinearSVC()
grid = GridSearchCV(lsvc,param_grid=param_grid,cv=5,scoring='neg_log_loss')
grid.fit(x_train,y_train)

ValueError: Unsupported set of arguments: The combination of penalty='l1' and loss='squared_hinge' are not supported when dual=True, Parameters: penalty='l1', loss='squared_hinge', dual=True

本来像仿照Logistic回归使用交叉验证来调参数，但出现了以上错误使我非常不解，想保留错误结果以供日后解答

In [25]:
#这条路行不通，所以只能自己写一个函数来循环
from sklearn.svm import LinearSVC
def linearsvc_param(C,x_train,y_train,x_test,y_test):
    lsvc = LinearSVC(C=C)
    lsvc.fit(x_train,y_train)
    print('当C={}时，测试集上的正确率为：{}'.format(C,lsvc.score(x_test,y_test)))
    return lsvc.score(x_test,y_test)

In [26]:
C = [0.1,1,10,100]
accuracys = []
for i,c in enumerate(C):
    accuracy = linearsvc_param(c,x_train,y_train,x_test,y_test)
    accuracys.append(accuracy)
    

当C=0.1时，测试集上的正确率为：0.7987012987012987
当C=1时，测试集上的正确率为：0.7857142857142857
当C=10时，测试集上的正确率为：0.7857142857142857
当C=100时，测试集上的正确率为：0.7532467532467533


想连着penalty一起调，没想到又不领我清，还是上面的错误，郁闷......只调C也是每次结果都不一样。总体上看，还是C=0.1时最好。这个分数也比Logistic回归的要高一些。可视化和进一步调参就略去了，被那个错误弄的心烦......

# SVC

In [31]:
#还是想尝试一下SVC，看看它和线性SVM的结果有什么差别
from sklearn.svm import SVC
def svc_param(C,x_train,y_train,x_test,y_test):
    svc = SVC(C=C)
    svc.fit(x_train,y_train)
    print('当C={}时，测试集上的正确率为：{}'.format(C,svc.score(x_test,y_test)))
    return svc.score(x_test,y_test)

In [34]:
C = [0.1,0.01,1,10,100]
accuracys = []
for i,c in enumerate(C):
    accuracy = svc_param(c,x_train,y_train,x_test,y_test)
    accuracys.append(accuracy)
    

当C=0.1时，测试集上的正确率为：0.7532467532467533
当C=0.01时，测试集上的正确率为：0.6753246753246753
当C=1时，测试集上的正确率为：0.7922077922077922
当C=10时，测试集上的正确率为：0.7402597402597403
当C=100时，测试集上的正确率为：0.6363636363636364


这个结果,emmmm.....还是用rbf核吧，顺便调一下γ参数

# 带RBF核的SVC

In [42]:
from sklearn.svm import SVC
def svc_rbf_param(C,gamma,x_train,y_train,x_test,y_test):
    svc_rbf = SVC(C=C,gamma=gamma)
    svc_rbf.fit(x_train,y_train)
    accuracy = svc_rbf.score(x_test,y_test)
    if(accuracy>0.75):
        print('当C={},gamma={}时，测试集上的正确率为：{}'.format(C,gamma,accuracy))
    return accuracy

In [43]:
C = [0.1,0.01,1,10,100]
gamma = [0.1,0.01,1,10,100]
accuracys = []
for i,c in enumerate(C):
    for j,g in enumerate(gamma):
        accuracy = svc_rbf_param(c,g,x_train,y_train,x_test,y_test)
        accuracys.append(accuracy)
    

当C=0.1,gamma=0.1时，测试集上的正确率为：0.7662337662337663
当C=1,gamma=0.1时，测试集上的正确率为：0.7857142857142857
当C=1,gamma=0.01时，测试集上的正确率为：0.7727272727272727
当C=10,gamma=0.1时，测试集上的正确率为：0.7727272727272727
当C=10,gamma=0.01时，测试集上的正确率为：0.8051948051948052
当C=100,gamma=0.01时，测试集上的正确率为：0.8051948051948052


为了不看的烟花，这里做了一个筛选，只看正确率高的几个，看起来，gamma越小的时候正确率越高

到这里，参数调优过程基本熟悉，具体步骤不再继续，等到将来有些造诣时还会回来再处理这些数据